# Windstorm project
This is a first out of several notebooks on Machine Learning-based prediction of severe surface winds associated with Extratropical Windstorms over different European geographical regions. Specifically, this study emphasizes how the temporal evolution characteristics of different storm internal and environmental predictors ("history") may contain useful information for quick evolution of severe wind potential overland.  

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd
import glob, os

# Add the path to the directory containing the module
import sys
sys.path.append('../../')
from util.ml import baseline, metrics

## Read data and trained models
In this notebook, we train a series of models with increasing complexity. To evaluate severe weather potential in a low sample size regime, we apply k-mean clustering to historical windstorm data and geospatial coordinates to identify coherent wind clusters within the European Union domain.
The severe wind potential of a particular windstorm can be defined by comparing the maximum near-surface wind of the storm after landfall in different clusters to their characteristics near-surface wind daily maxima climatologies. 

In [10]:
# Find the folder name organized by seed number
seed_docs = sorted(glob.glob('../../datas/seed_*/'))

data_dict = []
# Read files in a for loop
for seed_doc in seed_docs:
    # Read input data
    # Filter out the columns associated with convective_precipitation and large_scale_snowfall (model variables, not easily retrievable in observations)
    Xtrain_40 = baseline.filter_columns(pd.read_csv(seed_doc + 'X_train_40.csv'),"^(convective_precipitation_|large_scale_snowfall_)")
    Xtrain_30 = baseline.filter_columns(pd.read_csv(seed_doc + 'X_train_30.csv'),"^(convective_precipitation_|large_scale_snowfall_)")
    Xtrain_20 = baseline.filter_columns(pd.read_csv(seed_doc + 'X_train_20.csv'),"^(convective_precipitation_|large_scale_snowfall_)")
    Xvalid_40 = baseline.filter_columns(pd.read_csv(seed_doc + 'X_validation_40.csv'),"^(convective_precipitation_|large_scale_snowfall_)")
    Xvalid_30 = baseline.filter_columns(pd.read_csv(seed_doc + 'X_validation_30.csv'),"^(convective_precipitation_|large_scale_snowfall_)")
    Xvalid_20 = baseline.filter_columns(pd.read_csv(seed_doc + 'X_validation_20.csv'),"^(convective_precipitation_|large_scale_snowfall_)")

    # Read output data
    ytrain_cdf = pd.read_csv(seed_doc + 'y_train_cdf.csv')
    ytrain_max = pd.read_csv(seed_doc + 'y_train_max.csv')
    yvalid_cdf = pd.read_csv(seed_doc + 'y_validation_cdf.csv')
    yvalid_max = pd.read_csv(seed_doc + 'y_validation_max.csv')
    
    data = {
        'Xtrain_40': Xtrain_40,
        'Xtrain_30': Xtrain_30,
        'Xtrain_20': Xtrain_20,
        'Xvalid_40': Xvalid_40,
        'Xvalid_30': Xvalid_30,
        'Xvalid_20': Xvalid_20,
        'ytrain_cdf': ytrain_cdf,
        'ytrain_max': ytrain_max,
        'yvalid_cdf': yvalid_cdf,
        'yvalid_max': yvalid_max
    }
    data_dict.append(data)

# Find the name of the columns that we removed from the data
column_name_40 = pd.read_csv(seed_doc + 'X_train_40.csv').filter(regex="^(convective_precipitation_|large_scale_snowfall_)").columns
column_name_30 = pd.read_csv(seed_doc + 'X_train_30.csv').filter(regex="^(convective_precipitation_|large_scale_snowfall_)").columns
column_name_20 = pd.read_csv(seed_doc + 'X_train_20.csv').filter(regex="^(convective_precipitation_|large_scale_snowfall_)").columns

# All column indices
varname_40 = pd.read_csv(seed_doc + 'X_train_40.csv').columns
varname_30 = pd.read_csv(seed_doc + 'X_train_30.csv').columns
varname_20 = pd.read_csv(seed_doc + 'X_train_20.csv').columns

# Column indices for the removed variables
filtindex_40 = [list(varname_40).index(list(column_name_40)[i]) for i in range(len(list(column_name_40)))]
filtindex_30 = [list(varname_30).index(list(column_name_30)[i]) for i in range(len(list(column_name_30)))]
filtindex_20 = [list(varname_20).index(list(column_name_20)[i]) for i in range(len(list(column_name_20)))]

## Baseline models

### Linear Regression

In [11]:
# Train a linear regression model with the 20-most correlated inputs
linreg_20_cdf = []
linreg_20_max = []
for ind in range(len(data_dict)):
    lin_20_cdf = baseline.train_linear_regression(baseline.normalize_data(data_dict[ind]['Xtrain_20']), data_dict[ind]['ytrain_cdf'])
    lin_20_max = baseline.train_linear_regression(baseline.normalize_data(data_dict[ind]['Xtrain_20']), data_dict[ind]['ytrain_max'])
    linreg_20_cdf.append(lin_20_cdf)
    linreg_20_max.append(lin_20_max) 

# Train a linear regression model with the 30-most correlated inputs
linreg_30_cdf = []
linreg_30_max = []
for ind in range(len(data_dict)):
    lin_30_cdf = baseline.train_linear_regression(baseline.normalize_data(data_dict[ind]['Xtrain_30']), data_dict[ind]['ytrain_cdf'])
    lin_30_max = baseline.train_linear_regression(baseline.normalize_data(data_dict[ind]['Xtrain_30']), data_dict[ind]['ytrain_max'])
    linreg_30_cdf.append(lin_30_cdf)
    linreg_30_max.append(lin_30_max) 

# Train a linear regression model with the 40-most correlated inputs
linreg_40_cdf = []
linreg_40_max = []
for ind in range(len(data_dict)):
    lin_40_cdf = baseline.train_linear_regression(baseline.normalize_data(data_dict[ind]['Xtrain_40']), data_dict[ind]['ytrain_cdf'])
    lin_40_max = baseline.train_linear_regression(baseline.normalize_data(data_dict[ind]['Xtrain_40']), data_dict[ind]['ytrain_max'])
    linreg_40_cdf.append(lin_40_cdf)
    linreg_40_max.append(lin_40_max)

In [15]:
baseline.save_models(linreg_20_cdf, '../../datas/proc/part1/baseline_linreg_20_cdf.pkl')
baseline.save_models(linreg_20_max, '../../datas/proc/part1/baseline_linreg_20_max.pkl')
baseline.save_models(linreg_30_cdf, '../../datas/proc/part1/baseline_linreg_30_cdf.pkl')
baseline.save_models(linreg_30_max, '../../datas/proc/part1/baseline_linreg_30_max.pkl')
baseline.save_models(linreg_40_cdf, '../../datas/proc/part1/baseline_linreg_40_cdf.pkl')
baseline.save_models(linreg_40_max, '../../datas/proc/part1/baseline_linreg_40_max.pkl')

In [4]:
# Evaluate the linear regression model
performance_table_20 = (metrics.evaluate_model(linreg_20_cdf,linreg_20_max,data_dict,20))
performance_table_30 = (metrics.evaluate_model(linreg_30_cdf,linreg_30_max,data_dict,30))
performance_table_40 = (metrics.evaluate_model(linreg_40_cdf,linreg_40_max,data_dict,40))

# Create a table storing the r2 values
r2_table_20 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_20], 
    'train_max': [x['train_max']['R2'] for x in performance_table_20], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_20], 
    'val_max': [x['val_max']['R2'] for x in performance_table_20]}
    )

r2_table_30 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_30],
    'train_max': [x['train_max']['R2'] for x in performance_table_30],
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_30],
    'val_max': [x['val_max']['R2'] for x in performance_table_30]}
    )

r2_table_40 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_40],
    'train_max': [x['train_max']['R2'] for x in performance_table_40],
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_40],
    'val_max': [x['val_max']['R2'] for x in performance_table_40]}  
    )

# Create a table storing the RMSE values
rmse_table_20 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['RMSE'] for x in performance_table_20], 
    'train_max': [x['train_max']['RMSE'] for x in performance_table_20], 
    'val_cdf': [x['val_cdf']['RMSE'] for x in performance_table_20], 
    'val_max': [x['val_max']['RMSE'] for x in performance_table_20]}
    )

rmse_table_30 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['RMSE'] for x in performance_table_30], 
    'train_max': [x['train_max']['RMSE'] for x in performance_table_30], 
    'val_cdf': [x['val_cdf']['RMSE'] for x in performance_table_30], 
    'val_max': [x['val_max']['RMSE'] for x in performance_table_30]}
    )

rmse_table_40 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['RMSE'] for x in performance_table_40], 
    'train_max': [x['train_max']['RMSE'] for x in performance_table_40], 
    'val_cdf': [x['val_cdf']['RMSE'] for x in performance_table_40], 
    'val_max': [x['val_max']['RMSE'] for x in performance_table_40]}
    )

# Create a table storing the MAE values
mae_table_20 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['MAE'] for x in performance_table_20], 
    'train_max': [x['train_max']['MAE'] for x in performance_table_20], 
    'val_cdf': [x['val_cdf']['MAE'] for x in performance_table_20], 
    'val_max': [x['val_max']['MAE'] for x in performance_table_20]}
    )

mae_table_30 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['MAE'] for x in performance_table_30], 
    'train_max': [x['train_max']['MAE'] for x in performance_table_30], 
    'val_cdf': [x['val_cdf']['MAE'] for x in performance_table_30], 
    'val_max': [x['val_max']['MAE'] for x in performance_table_30]}
    )

mae_table_40 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['MAE'] for x in performance_table_40], 
    'train_max': [x['train_max']['MAE'] for x in performance_table_40], 
    'val_cdf': [x['val_cdf']['MAE'] for x in performance_table_40], 
    'val_max': [x['val_max']['MAE'] for x in performance_table_40]}
    )

In [5]:
r2analy_table_20 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['r2analysis'] for x in performance_table_20], 
    'train_max': [x['train_max']['r2analysis'] for x in performance_table_20], 
    'val_cdf': [x['val_cdf']['r2analysis'] for x in performance_table_20], 
    'val_max': [x['val_max']['r2analysis'] for x in performance_table_20]}
    )

In [8]:
r2analy_table_20['val_cdf'].values[0

array([{'sum_of_error': 2217.243655897847, 'variance': 302.0175593389944},
       {'sum_of_error': 1234.9945016272798, 'variance': 188.9299142518861},
       {'sum_of_error': 1874.7728148543408, 'variance': 396.91335623509684},
       {'sum_of_error': 1367.9827876927154, 'variance': 247.5818987570473},
       {'sum_of_error': 1333.000849825739, 'variance': 352.5103045287219},
       {'sum_of_error': 1048.6096605187165, 'variance': 204.4021459866127},
       {'sum_of_error': 1540.273717222057, 'variance': 165.4057080760094},
       {'sum_of_error': 2397.4232944820906, 'variance': 210.85454807805274},
       {'sum_of_error': 1197.4074229630887, 'variance': 348.21439066089033},
       {'sum_of_error': 2611.1769738831235, 'variance': 322.2458208311451}],
      dtype=object)

In the low data sample regime, it is clear that all the baseline models have an overfitting problem. Take a look at the models trained with 20-most correlated variables and their PCs

In [5]:
r2_table_20

,train_cdf,train_max,val_cdf,val_max
0,0.896629,0.928229,-6.341440,-5.166038
1,0.873582,0.905884,-5.536786,-2.633175
2,0.853346,0.888714,-3.723381,-2.546466
3,0.874684,0.909443,-4.525375,-3.688000
4,0.885469,0.908917,-2.781452,-2.172883
5,0.864496,0.896081,-4.130130,-2.610774
6,0.844796,0.880227,-8.312095,-4.086494
7,0.885900,0.911460,-10.370034,-7.290632
8,0.887876,0.917577,-2.438707,-1.892122
9,0.894406,0.923712,-7.103059,-4.955472


30-most correlated variables and their PCs

In [6]:
r2_table_30

,train_cdf,train_max,val_cdf,val_max
0,1.0,1.0,-15.626446,-9.143537
1,1.0,1.0,-15.074327,-7.185135
2,1.0,1.0,-23.418424,-15.506171
3,1.0,1.0,-15.312413,-10.534606
4,1.0,1.0,-6.693788,-6.104743
5,1.0,1.0,-16.273791,-8.324698
6,1.0,1.0,-27.480188,-12.522046
7,1.0,1.0,-21.669072,-11.271964
8,1.0,1.0,-8.257546,-5.476506
9,1.0,1.0,-29.269378,-15.696830


### Random Forest

In [19]:
from tqdm import tqdm
# Train a RF model with the 20-most correlated inputs
rf_20_cdf = []
rf_20_max = []
for ind in tqdm(range(len(data_dict))):
    rf_20_cdfz = baseline.train_rf(data_dict[ind]['Xtrain_20'], data_dict[ind]['ytrain_cdf'],42)
    rf_20_maxz = baseline.train_rf(data_dict[ind]['Xtrain_20'], data_dict[ind]['ytrain_max'],42)
    rf_20_cdf.append(rf_20_cdfz)
    rf_20_max.append(rf_20_maxz) 

# Train a RF model with the 30-most correlated inputs
rf_30_cdf = []
rf_30_max = []
for ind in tqdm(range(len(data_dict))):
    rf_30_cdfz = baseline.train_rf(data_dict[ind]['Xtrain_30'], data_dict[ind]['ytrain_cdf'],42)
    rf_30_maxz = baseline.train_rf(data_dict[ind]['Xtrain_30'], data_dict[ind]['ytrain_max'],42)
    rf_30_cdf.append(rf_30_cdfz)
    rf_30_max.append(rf_30_maxz)

# Train a RF model with the 40-most correlated inputs
rf_40_cdf = []
rf_40_max = []
for ind in tqdm(range(len(data_dict))):
    rf_40_cdfz = baseline.train_rf(data_dict[ind]['Xtrain_40'], data_dict[ind]['ytrain_cdf'],42)
    rf_40_maxz = baseline.train_rf(data_dict[ind]['Xtrain_40'], data_dict[ind]['ytrain_max'],42)
    rf_40_cdf.append(rf_40_cdfz)
    rf_40_max.append(rf_40_maxz)

 10%|█         | 1/10 [00:33<05:00, 33.40s/it]


KeyboardInterrupt: 

In [78]:
baseline.save_models(rf_20_cdf, '../../datas/proc/part1/baseline_rf_20_cdf.pkl')
baseline.save_models(rf_20_max, '../../datas/proc/part1/baseline_rf_20_max.pkl')
baseline.save_models(rf_30_cdf, '../../datas/proc/part1/baseline_rf_30_cdf.pkl')
baseline.save_models(rf_30_max, '../../datas/proc/part1/baseline_rf_30_max.pkl')
baseline.save_models(rf_40_cdf, '../../datas/proc/part1/baseline_rf_40_cdf.pkl')
baseline.save_models(rf_40_max, '../../datas/proc/part1/baseline_rf_40_max.pkl')

In [10]:
# Evaluate the Random Forest model
rf_performance_table_20 = (metrics.evaluate_model(rf_20_cdf,rf_20_max,data_dict,20))
rf_performance_table_30 = (metrics.evaluate_model(rf_30_cdf,rf_30_max,data_dict,30))
rf_performance_table_40 = (metrics.evaluate_model(rf_40_cdf,rf_40_max,data_dict,40))

# Create a table storing the r2 values
rf_r2_table_20 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_20], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_20], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_20], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_20]}
    )

rf_r2_table_30 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_30],
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_30],
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_30],
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_30]}
    )

rf_r2_table_40 = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_40],
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_40],
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_40],
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_40]}  
    )

Sensitivity experiment: Incorporate nonlinearity by using Random Forest

In [14]:
rf_r2_table_20

,train_cdf,train_max,val_cdf,val_max
0,0.125198,0.126835,-1.779630,-0.578028
1,0.119736,0.130174,-1.209389,-0.511947
2,0.405908,0.129667,-0.165449,-0.107460
3,0.126871,0.133974,-0.586515,-0.271364
4,0.113250,0.132030,-0.106133,-0.180633
5,0.107411,0.139727,-0.619027,-0.522737
6,0.114706,0.130970,-5.104729,-1.290778
7,0.113581,0.131548,-0.202503,-0.354429
8,0.122875,0.470241,-0.132141,-0.113430
9,0.114009,0.442240,-0.235993,-0.350141


In [15]:
rf_r2_table_30

,train_cdf,train_max,val_cdf,val_max
0,0.146923,0.134286,-1.112254,-0.523230
1,0.128838,0.137175,-0.482713,-0.418805
2,0.140519,0.151934,-0.224490,-0.380391
3,0.144362,0.144407,-1.110380,-0.457091
4,0.128156,0.144961,-0.166800,-0.215663
5,0.126519,0.140871,-0.679359,-0.492123
6,0.130437,0.143217,-5.782299,-1.314509
7,0.127362,0.139184,-0.203001,-0.344268
8,0.143165,0.145314,-0.145927,-0.151131
9,0.142522,0.136762,-0.258481,-0.302503


In [13]:
rf_r2_table_40

,train_cdf,train_max,val_cdf,val_max
0,0.142035,0.131455,-0.854148,-0.492354
1,0.132687,0.134115,-0.499831,-0.430950
2,0.140864,0.156946,-0.185747,-0.310611
3,0.157041,0.141237,-1.042502,-0.415374
4,0.137127,0.145056,-0.196723,-0.227618
5,0.139410,0.143695,-0.596939,-0.455233
6,0.132352,0.141793,-5.586163,-1.238528
7,0.136406,0.144586,-0.232322,-0.372766
8,0.136380,0.146682,-0.143709,-0.134099
9,0.151950,0.138065,-0.257543,-0.306455


## Feature selection with XGBoost

In [7]:
# Find the folder name organized by seed number
seed_docs = sorted(glob.glob('../../datas/seed_*/'))

model_dict = []
# Read files in a for loop
for seed_doc in seed_docs:
    model_XGB_cdf_20 = pickle.load(open(seed_doc + 'model_xgboost/model_cdf_20.pkl', 'rb'))
    model_XGB_cdf_30 = pickle.load(open(seed_doc + 'model_xgboost/model_cdf_30.pkl', 'rb'))
    model_XGB_cdf_40 = pickle.load(open(seed_doc + 'model_xgboost/model_cdf_40.pkl', 'rb'))

    model_XGB_max_20 = pickle.load(open(seed_doc + 'model_xgboost/model_max_20.pkl', 'rb'))
    model_XGB_max_30 = pickle.load(open(seed_doc + 'model_xgboost/model_max_30.pkl', 'rb'))
    model_XGB_max_40 = pickle.load(open(seed_doc + 'model_xgboost/model_max_40.pkl', 'rb'))

    model_dict.append({
        'model_XGB_cdf_20': model_XGB_cdf_20,
        'model_XGB_cdf_30': model_XGB_cdf_30,
        'model_XGB_cdf_40': model_XGB_cdf_40,
        'model_XGB_max_20': model_XGB_max_20,
        'model_XGB_max_30': model_XGB_max_30,
        'model_XGB_max_40': model_XGB_max_40
    })

/Users/itam/miniconda3/envs/myenv/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [11:50:10] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [8]:
def filt_index_no_conv_snow(index_40,index_30,index_20,filtindex_40,filtindex_30,filtindex_20):
    filt_index_40 = []
    for obj in index_40:
        if obj in filtindex_40:
            continue
        else:
            if obj>39 and obj<85:
                filt_index_40.append(obj-2)
            elif obj>85:
                filt_index_40.append(obj-5)
            else:
                filt_index_40.append(obj)

    filt_index_30 = []
    for obj in index_30:
        if obj in filtindex_30:
            continue
        else:
            if obj>39:
                filt_index_30.append(obj-2)
            else:
                filt_index_30.append(obj)

    filt_index_20 = []
    for obj in index_20:
        if obj in filtindex_20:
            continue
        else:
            if obj>39:
                filt_index_20.append(obj-2)
            else:
                filt_index_20.append(obj)
    return filt_index_20, filt_index_30, filt_index_40

In [9]:
# Find the feature importance of the RF model
feature_importance_20_cdf_xgb = []
feature_importance_30_cdf_xgb = []
feature_importance_40_cdf_xgb = []
feature_importance_20_max_xgb = []
feature_importance_30_max_xgb = []
feature_importance_40_max_xgb = []

for ind in range(len(model_dict)):
    feature_importance_20_cdf_xgb.append(model_dict[ind]['model_XGB_cdf_20'].feature_importances_)
    feature_importance_30_cdf_xgb.append(model_dict[ind]['model_XGB_cdf_30'].feature_importances_)
    feature_importance_40_cdf_xgb.append(model_dict[ind]['model_XGB_cdf_40'].feature_importances_)
    feature_importance_20_max_xgb.append(model_dict[ind]['model_XGB_max_20'].feature_importances_)
    feature_importance_30_max_xgb.append(model_dict[ind]['model_XGB_max_30'].feature_importances_)
    feature_importance_40_max_xgb.append(model_dict[ind]['model_XGB_max_40'].feature_importances_)

# Find index where feature importance is greater than 0
index_20_cdf_xgb = [[ind for ind, x in enumerate(feature_importance_20_cdf_xgb[i]) if x > 0] for i in range(len(feature_importance_20_cdf_xgb))]
index_30_cdf_xgb = [[ind for ind, x in enumerate(feature_importance_30_cdf_xgb[i]) if x > 0] for i in range(len(feature_importance_30_cdf_xgb))]
index_40_cdf_xgb = [[ind for ind, x in enumerate(feature_importance_40_cdf_xgb[i]) if x > 0] for i in range(len(feature_importance_40_cdf_xgb))]
index_20_max_xgb = [[ind for ind, x in enumerate(feature_importance_20_max_xgb[i]) if x > 0] for i in range(len(feature_importance_20_max_xgb))]
index_30_max_xgb = [[ind for ind, x in enumerate(feature_importance_30_max_xgb[i]) if x > 0] for i in range(len(feature_importance_30_max_xgb))]
index_40_max_xgb = [[ind for ind, x in enumerate(feature_importance_40_max_xgb[i]) if x > 0] for i in range(len(feature_importance_40_max_xgb))]

# Process the index to remove the columns associated with convective_precipitation and large_scale_snowfall
index_20_filt_cdf_xgb = [filt_index_no_conv_snow(index_40_cdf_xgb[i],index_30_cdf_xgb[i],index_20_cdf_xgb[i],filtindex_40, filtindex_30, filtindex_20)[0] for i in range(len(index_20_cdf_xgb))]
index_30_filt_cdf_xgb = [filt_index_no_conv_snow(index_40_cdf_xgb[i],index_30_cdf_xgb[i],index_20_cdf_xgb[i],filtindex_40, filtindex_30, filtindex_20)[1] for i in range(len(index_30_cdf_xgb))]
index_40_filt_cdf_xgb = [filt_index_no_conv_snow(index_40_cdf_xgb[i],index_30_cdf_xgb[i],index_20_cdf_xgb[i],filtindex_40, filtindex_30, filtindex_20)[2] for i in range(len(index_40_cdf_xgb))]
index_20_filt_max_xgb = [filt_index_no_conv_snow(index_40_max_xgb[i],index_30_max_xgb[i],index_20_max_xgb[i],filtindex_40, filtindex_30, filtindex_20)[0] for i in range(len(index_20_max_xgb))]
index_30_filt_max_xgb = [filt_index_no_conv_snow(index_40_max_xgb[i],index_30_max_xgb[i],index_20_max_xgb[i],filtindex_40, filtindex_30, filtindex_20)[1] for i in range(len(index_30_max_xgb))]
index_40_filt_max_xgb = [filt_index_no_conv_snow(index_40_max_xgb[i],index_30_max_xgb[i],index_20_max_xgb[i],filtindex_40, filtindex_30, filtindex_20)[2] for i in range(len(index_40_max_xgb))]
            
# Filter the input data with the feature importance
data_dict_filt_cdf_xgb = [baseline.filt_with_feature_importance(index_20_filt_cdf_xgb[ind], index_30_filt_cdf_xgb[ind], index_40_filt_cdf_xgb[ind], data_dict[ind]) for ind in range(len(data_dict))]
data_dict_filt_max_xgb = [baseline.filt_with_feature_importance(index_20_filt_max_xgb[ind], index_30_filt_max_xgb[ind], index_40_filt_max_xgb[ind], data_dict[ind]) for ind in range(len(data_dict))]  

### Linear Regression

In [10]:
# Train a linear regression model to predict CDF with the filtered data 
lin_20_cdf_filt_xgb = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_cdf_xgb[i]['Xtrain_20']), data_dict_filt_cdf_xgb[i]['ytrain_cdf']) for i in range(len(data_dict_filt_cdf_xgb))]
lin_30_cdf_filt_xgb = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_cdf_xgb[i]['Xtrain_30']), data_dict_filt_cdf_xgb[i]['ytrain_cdf']) for i in range(len(data_dict_filt_cdf_xgb))]
lin_40_cdf_filt_xgb = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_cdf_xgb[i]['Xtrain_40']), data_dict_filt_cdf_xgb[i]['ytrain_cdf']) for i in range(len(data_dict_filt_cdf_xgb))]

# Train a linear regression model to predict max with the filtered data
lin_20_max_filt_xgb = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_max_xgb[i]['Xtrain_20']), data_dict_filt_max_xgb[i]['ytrain_max']) for i in range(len(data_dict_filt_max_xgb))]
lin_30_max_filt_xgb = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_max_xgb[i]['Xtrain_30']), data_dict_filt_max_xgb[i]['ytrain_max']) for i in range(len(data_dict_filt_max_xgb))]
lin_40_max_filt_xgb = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_max_xgb[i]['Xtrain_40']), data_dict_filt_max_xgb[i]['ytrain_max']) for i in range(len(data_dict_filt_max_xgb))]

# Evaluate the model
performance_table_20_filt_xgb_cdf = metrics.evaluate_cdfmodel(lin_20_cdf_filt_xgb, data_dict_filt_cdf_xgb, 20)
performance_table_30_filt_xgb_cdf = metrics.evaluate_cdfmodel(lin_30_cdf_filt_xgb, data_dict_filt_cdf_xgb, 30)
performance_table_40_filt_xgb_cdf = metrics.evaluate_cdfmodel(lin_40_cdf_filt_xgb, data_dict_filt_cdf_xgb, 40)
performance_table_20_filt_xgb_max = metrics.evaluate_maxmodel(lin_20_max_filt_xgb, data_dict_filt_max_xgb, 20)
performance_table_30_filt_xgb_max = metrics.evaluate_maxmodel(lin_30_max_filt_xgb, data_dict_filt_max_xgb, 30)
performance_table_40_filt_xgb_max = metrics.evaluate_maxmodel(lin_40_max_filt_xgb, data_dict_filt_max_xgb, 40)

In [79]:
baseline.save_models(lin_20_cdf_filt_xgb, '../../datas/proc/part1/xgb_linreg_20_cdf.pkl')
baseline.save_models(lin_30_cdf_filt_xgb, '../../datas/proc/part1/xgb_linreg_30_cdf.pkl')
baseline.save_models(lin_40_cdf_filt_xgb, '../../datas/proc/part1/xgb_linreg_40_cdf.pkl')
baseline.save_models(lin_20_max_filt_xgb, '../../datas/proc/part1/xgb_linreg_20_max.pkl')
baseline.save_models(lin_30_max_filt_xgb, '../../datas/proc/part1/xgb_linreg_30_max.pkl')
baseline.save_models(lin_40_max_filt_xgb, '../../datas/proc/part1/xgb_linreg_40_max.pkl')

In [11]:
# Create a table storing the r2 values
r2_table_20_xgb = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_20_filt_xgb_cdf], 
    'train_max': [x['train_max']['R2'] for x in performance_table_20_filt_xgb_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_20_filt_xgb_cdf], 
    'val_max': [x['val_max']['R2'] for x in performance_table_20_filt_xgb_max]}
    )

r2_table_30_xgb = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_30_filt_xgb_cdf], 
    'train_max': [x['train_max']['R2'] for x in performance_table_30_filt_xgb_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_30_filt_xgb_cdf], 
    'val_max': [x['val_max']['R2'] for x in performance_table_30_filt_xgb_max]}
    )

r2_table_40_xgb = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_40_filt_xgb_cdf], 
    'train_max': [x['train_max']['R2'] for x in performance_table_40_filt_xgb_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_40_filt_xgb_cdf], 
    'val_max': [x['val_max']['R2'] for x in performance_table_40_filt_xgb_max]}
    )

In [12]:
r2_table_20_xgb

,train_cdf,train_max,val_cdf,val_max
0,0.759730,0.772760,-1.789951,-0.114932
1,0.671302,0.759953,-1.130568,-0.465765
2,0.724634,0.808947,-1.544458,-1.224188
3,0.731632,0.788931,-2.277185,-1.025317
4,0.676754,0.845012,-0.903179,-1.318035
5,0.696185,0.760897,-0.771759,-0.262673
6,0.623637,0.812579,0.044108,-2.197147
7,0.665793,0.803565,-0.930111,-0.826961
8,0.652062,0.727650,-0.474182,-0.066106
9,0.737505,0.846945,-1.418323,-0.963630


In [13]:
r2_table_30_xgb

,train_cdf,train_max,val_cdf,val_max
0,1.000000,1.000000,-42.365077,-21.130787
1,1.000000,0.969460,-11.437523,-21.541297
2,0.837099,0.875103,-2.078987,-3.690126
3,0.664907,0.949048,-0.832215,-10.235444
4,1.000000,0.924288,-6.450378,-17.249864
5,0.960380,0.892787,-19.460591,-3.701572
6,0.828335,0.958776,-10.503599,-16.374274
7,0.919141,0.979604,-13.045015,-40.127148
8,0.732350,0.917981,-0.586173,-3.140291
9,0.873809,0.988769,-4.884529,-58.207729


### Random Forest

In [75]:
# Train a RF model to predict CDF with the filtered data 
rf_20_cdf_filt_xgb = [baseline.train_rf(data_dict_filt_cdf_xgb[i]['Xtrain_20'], data_dict_filt_cdf_xgb[i]['ytrain_cdf'], 42) for i in tqdm(range(len(data_dict_filt_cdf_xgb)))]
rf_30_cdf_filt_xgb = [baseline.train_rf(data_dict_filt_cdf_xgb[i]['Xtrain_30'], data_dict_filt_cdf_xgb[i]['ytrain_cdf'], 42) for i in tqdm(range(len(data_dict_filt_cdf_xgb)))]
rf_40_cdf_filt_xgb = [baseline.train_rf(data_dict_filt_cdf_xgb[i]['Xtrain_40'], data_dict_filt_cdf_xgb[i]['ytrain_cdf'], 42) for i in tqdm(range(len(data_dict_filt_cdf_xgb)))]

# Train a RF model to predict max with the filtered data
rf_20_max_filt_xgb = [baseline.train_rf(data_dict_filt_max_xgb[i]['Xtrain_20'], data_dict_filt_max_xgb[i]['ytrain_max'], 42) for i in tqdm(range(len(data_dict_filt_max_xgb)))]
rf_30_max_filt_xgb = [baseline.train_rf(data_dict_filt_max_xgb[i]['Xtrain_30'], data_dict_filt_max_xgb[i]['ytrain_max'], 42) for i in tqdm(range(len(data_dict_filt_max_xgb)))]
rf_40_max_filt_xgb = [baseline.train_rf(data_dict_filt_max_xgb[i]['Xtrain_40'], data_dict_filt_max_xgb[i]['ytrain_max'], 42) for i in tqdm(range(len(data_dict_filt_max_xgb)))]

100%|██████████| 10/10 [01:39<00:00,  9.98s/it]


In [80]:
baseline.save_models(rf_20_cdf_filt_xgb, '../../datas/proc/part1/xgb_rf_20_cdf.pkl')
baseline.save_models(rf_30_cdf_filt_xgb, '../../datas/proc/part1/xgb_rf_30_cdf.pkl')
baseline.save_models(rf_40_cdf_filt_xgb, '../../datas/proc/part1/xgb_rf_40_cdf.pkl')
baseline.save_models(rf_20_max_filt_xgb, '../../datas/proc/part1/xgb_rf_20_max.pkl')
baseline.save_models(rf_30_max_filt_xgb, '../../datas/proc/part1/xgb_rf_30_max.pkl')
baseline.save_models(rf_40_max_filt_xgb, '../../datas/proc/part1/xgb_rf_40_max.pkl')

In [83]:
# Evaluate the model
rf_performance_table_20_filt_xgb_cdf = metrics.evaluate_cdfmodel(rf_20_cdf_filt_xgb, data_dict_filt_cdf_xgb, 20)
rf_performance_table_30_filt_xgb_cdf = metrics.evaluate_cdfmodel(rf_30_cdf_filt_xgb, data_dict_filt_cdf_xgb, 30)
rf_performance_table_40_filt_xgb_cdf = metrics.evaluate_cdfmodel(rf_40_cdf_filt_xgb, data_dict_filt_cdf_xgb, 40)
rf_performance_table_20_filt_xgb_max = metrics.evaluate_maxmodel(rf_20_max_filt_xgb, data_dict_filt_max_xgb, 20)
rf_performance_table_30_filt_xgb_max = metrics.evaluate_maxmodel(rf_30_max_filt_xgb, data_dict_filt_max_xgb, 30)
rf_performance_table_40_filt_xgb_max = metrics.evaluate_maxmodel(rf_40_max_filt_xgb, data_dict_filt_max_xgb, 40)

In [84]:
# Create a table storing the r2 values
rf_r2_table_20_xgb = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_20_filt_xgb_cdf], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_20_filt_xgb_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_20_filt_xgb_cdf], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_20_filt_xgb_max]}
    )

rf_r2_table_30_xgb = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_30_filt_xgb_cdf], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_30_filt_xgb_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_30_filt_xgb_cdf], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_30_filt_xgb_max]}
    )

rf_r2_table_40_xgb = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_40_filt_xgb_cdf], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_40_filt_xgb_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_40_filt_xgb_cdf], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_40_filt_xgb_max]}
    )

In [85]:
rf_r2_table_20_xgb

,train_cdf,train_max,val_cdf,val_max
0,0.125087,0.125902,-1.779630,-0.616098
1,0.113101,0.129722,-0.632175,-0.506543
2,0.410598,0.429612,-0.077461,-0.201473
3,0.127072,0.134389,-0.586728,-0.312970
4,0.113710,0.131753,-0.100798,-0.179701
5,0.109651,0.398565,-0.596304,-0.656428
6,0.405759,0.130970,-6.380835,-1.316224
7,0.411988,0.131396,-0.313987,-0.352116
8,0.125503,0.646834,-0.176320,-0.164300
9,0.114009,0.442911,-0.235993,-0.353471


In [86]:
rf_r2_table_30_xgb

,train_cdf,train_max,val_cdf,val_max
0,0.129411,0.133704,-0.855237,-0.520839
1,0.129118,0.130844,-0.515697,-0.429697
2,0.123242,0.137737,-0.038279,-0.071429
3,0.142018,0.143031,-1.244302,-0.419293
4,0.130531,0.147129,-0.166974,-0.240019
5,0.397227,0.142668,-0.829409,-0.527886
6,0.119049,0.141056,-5.737155,-1.304494
7,0.124548,0.137158,-0.173496,-0.332459
8,0.132369,0.145025,-0.136752,-0.163256
9,0.136380,0.132413,-0.236399,-0.294842


In [87]:
rf_r2_table_40_xgb

,train_cdf,train_max,val_cdf,val_max
0,0.128520,0.132253,-0.849520,-0.491262
1,0.129084,0.134414,-0.514496,-0.423288
2,0.123965,0.139400,-0.051073,-0.062540
3,0.149319,0.145457,-0.993310,-0.436676
4,0.138280,0.491059,-0.183483,-0.304093
5,0.140242,0.145400,-0.599211,-0.477250
6,0.148855,0.139515,-8.472444,-1.241013
7,0.129542,0.461183,-0.245354,-0.623167
8,0.127796,0.146379,-0.147709,-0.140100
9,0.151950,0.137668,-0.257543,-0.308336


## Feature selection with Random Forest

In [4]:
# Find the folder name organized by seed number
seed_docs = sorted(glob.glob('../../datas/seed_*/'))

model_dict = []
# Read files in a for loop
for seed_doc in seed_docs:
    model_RF_cdf_20 = pickle.load(open(seed_doc + 'model_random_forest/model_cdf_20.pkl', 'rb'))
    model_RF_cdf_30 = pickle.load(open(seed_doc + 'model_random_forest/model_cdf_30.pkl', 'rb'))
    model_RF_cdf_40 = pickle.load(open(seed_doc + 'model_random_forest/model_cdf_40.pkl', 'rb'))

    model_RF_max_20 = pickle.load(open(seed_doc + 'model_random_forest/model_max_20.pkl', 'rb'))
    model_RF_max_30 = pickle.load(open(seed_doc + 'model_random_forest/model_max_30.pkl', 'rb'))
    model_RF_max_40 = pickle.load(open(seed_doc + 'model_random_forest/model_max_40.pkl', 'rb'))

    model_dict.append({
        'model_RF_cdf_20': model_RF_cdf_20,
        'model_RF_cdf_30': model_RF_cdf_30,
        'model_RF_cdf_40': model_RF_cdf_40,
        'model_RF_max_20': model_RF_max_20,
        'model_RF_max_30': model_RF_max_30,
        'model_RF_max_40': model_RF_max_40
    })

/Users/itam/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/itam/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Find the feature importance of the RF model
feature_importance_20_cdf = []
feature_importance_30_cdf = []
feature_importance_40_cdf = []
feature_importance_20_max = []
feature_importance_30_max = []
feature_importance_40_max = []

for ind in range(len(model_dict)):
    feature_importance_20_cdf.append(model_dict[ind]['model_RF_cdf_20'].feature_importances_)
    feature_importance_30_cdf.append(model_dict[ind]['model_RF_cdf_30'].feature_importances_)
    feature_importance_40_cdf.append(model_dict[ind]['model_RF_cdf_40'].feature_importances_)
    feature_importance_20_max.append(model_dict[ind]['model_RF_max_20'].feature_importances_)
    feature_importance_30_max.append(model_dict[ind]['model_RF_max_30'].feature_importances_)
    feature_importance_40_max.append(model_dict[ind]['model_RF_max_40'].feature_importances_)

# Find index where feature importance is greater than 0
index_20_cdf = [[ind for ind, x in enumerate(feature_importance_20_cdf[i]) if x > 0] for i in range(len(feature_importance_20_cdf))]
index_30_cdf = [[ind for ind, x in enumerate(feature_importance_30_cdf[i]) if x > 0] for i in range(len(feature_importance_30_cdf))]
index_40_cdf = [[ind for ind, x in enumerate(feature_importance_40_cdf[i]) if x > 0] for i in range(len(feature_importance_40_cdf))]
index_20_max = [[ind for ind, x in enumerate(feature_importance_20_max[i]) if x > 0] for i in range(len(feature_importance_20_max))]
index_30_max = [[ind for ind, x in enumerate(feature_importance_30_max[i]) if x > 0] for i in range(len(feature_importance_30_max))]
index_40_max = [[ind for ind, x in enumerate(feature_importance_40_max[i]) if x > 0] for i in range(len(feature_importance_40_max))]

# Process the index to remove the columns associated with convective_precipitation and large_scale_snowfall
index_20_filt_cdf = [filt_index_no_conv_snow(index_40_cdf[i],index_30_cdf[i],index_20_cdf[i],filtindex_40, filtindex_30, filtindex_20)[0] for i in range(len(index_20_cdf))]
index_30_filt_cdf = [filt_index_no_conv_snow(index_40_cdf[i],index_30_cdf[i],index_20_cdf[i],filtindex_40, filtindex_30, filtindex_20)[1] for i in range(len(index_30_cdf))]
index_40_filt_cdf = [filt_index_no_conv_snow(index_40_cdf[i],index_30_cdf[i],index_20_cdf[i],filtindex_40, filtindex_30, filtindex_20)[2] for i in range(len(index_40_cdf))]
index_20_filt_max = [filt_index_no_conv_snow(index_40_max[i],index_30_max[i],index_20_max[i],filtindex_40, filtindex_30, filtindex_20)[0] for i in range(len(index_20_max))]
index_30_filt_max = [filt_index_no_conv_snow(index_40_max[i],index_30_max[i],index_20_max[i],filtindex_40, filtindex_30, filtindex_20)[1] for i in range(len(index_30_max))]
index_40_filt_max = [filt_index_no_conv_snow(index_40_max[i],index_30_max[i],index_20_max[i],filtindex_40, filtindex_30, filtindex_20)[2] for i in range(len(index_40_max))]
            
# Filter the input data with the feature importance
data_dict_filt_cdf_rf = [baseline.filt_with_feature_importance(index_20_filt_cdf[ind], index_30_filt_cdf[ind], index_40_filt_cdf[ind], data_dict[ind]) for ind in range(len(data_dict))]
data_dict_filt_max_rf = [baseline.filt_with_feature_importance(index_20_filt_max[ind], index_30_filt_max[ind], index_40_filt_max[ind], data_dict[ind]) for ind in range(len(data_dict))]  

In [10]:
data_dict_filt_cdf_rf[2]['Xtrain_20'].columns

Index(['10m_u_component_of_wind_std_PCA_2', 'surface_pressure_mean_PCA_2',
       '10m_v_component_of_wind_max_PCA_1',
       'mean_sea_level_pressure_max_PCA_2', 'surface_pressure_max_PCA_1',
       'surface_pressure_max_PCA_2', 'geopotential_1000_std_PCA_2',
       '2m_dewpoint_temperature_std_PCA_2',
       '2m_dewpoint_temperature_std_PCA_3'],
      dtype='object')

### MLR

In [49]:
# Train a linear regression model to predict CDF with the filtered data 
lin_20_cdf_filt_rf = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_cdf_rf[i]['Xtrain_20']), data_dict_filt_cdf_rf[i]['ytrain_cdf']) for i in range(len(data_dict_filt_cdf_rf))]
lin_30_cdf_filt_rf = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_cdf_rf[i]['Xtrain_30']), data_dict_filt_cdf_rf[i]['ytrain_cdf']) for i in range(len(data_dict_filt_cdf_rf))]
lin_40_cdf_filt_rf = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_cdf_rf[i]['Xtrain_40']), data_dict_filt_cdf_rf[i]['ytrain_cdf']) for i in range(len(data_dict_filt_cdf_rf))]

# Train a linear regression model to predict max with the filtered data
lin_20_max_filt_rf = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_max_rf[i]['Xtrain_20']), data_dict_filt_max_rf[i]['ytrain_max']) for i in range(len(data_dict_filt_max_rf))]
lin_30_max_filt_rf = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_max_rf[i]['Xtrain_30']), data_dict_filt_max_rf[i]['ytrain_max']) for i in range(len(data_dict_filt_max_rf))]
lin_40_max_filt_rf = [baseline.train_linear_regression(baseline.normalize_data(data_dict_filt_max_rf[i]['Xtrain_40']), data_dict_filt_max_rf[i]['ytrain_max']) for i in range(len(data_dict_filt_max_rf))]

# Evaluate the model
performance_table_20_filt_rf_cdf = metrics.evaluate_cdfmodel(lin_20_cdf_filt_rf, data_dict_filt_cdf_rf, 20)
performance_table_30_filt_rf_cdf = metrics.evaluate_cdfmodel(lin_30_cdf_filt_rf, data_dict_filt_cdf_rf, 30)
performance_table_40_filt_rf_cdf = metrics.evaluate_cdfmodel(lin_40_cdf_filt_rf, data_dict_filt_cdf_rf, 40)
performance_table_20_filt_rf_max = metrics.evaluate_maxmodel(lin_20_max_filt_rf, data_dict_filt_max_rf, 20)
performance_table_30_filt_rf_max = metrics.evaluate_maxmodel(lin_30_max_filt_rf, data_dict_filt_max_rf, 30)
performance_table_40_filt_rf_max = metrics.evaluate_maxmodel(lin_40_max_filt_rf, data_dict_filt_max_rf, 40)

In [81]:
baseline.save_models(lin_20_cdf_filt_rf, '../../datas/proc/part1/rf_linreg_20_cdf.pkl')
baseline.save_models(lin_30_cdf_filt_rf, '../../datas/proc/part1/rf_linreg_30_cdf.pkl')
baseline.save_models(lin_40_cdf_filt_rf, '../../datas/proc/part1/rf_linreg_40_cdf.pkl')
baseline.save_models(lin_20_max_filt_rf, '../../datas/proc/part1/rf_linreg_20_max.pkl')
baseline.save_models(lin_30_max_filt_rf, '../../datas/proc/part1/rf_linreg_30_max.pkl')
baseline.save_models(lin_40_max_filt_rf, '../../datas/proc/part1/rf_linreg_40_max.pkl')

In [50]:
# Create a table storing the r2 values
r2_table_20_rf = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_20_filt_rf_cdf], 
    'train_max': [x['train_max']['R2'] for x in performance_table_20_filt_rf_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_20_filt_rf_cdf], 
    'val_max': [x['val_max']['R2'] for x in performance_table_20_filt_rf_max]}
    )

r2_table_30_rf = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_30_filt_rf_cdf], 
    'train_max': [x['train_max']['R2'] for x in performance_table_30_filt_rf_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_30_filt_rf_cdf], 
    'val_max': [x['val_max']['R2'] for x in performance_table_30_filt_rf_max]}
    )

r2_table_40_rf = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in performance_table_40_filt_rf_cdf], 
    'train_max': [x['train_max']['R2'] for x in performance_table_40_filt_rf_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in performance_table_40_filt_rf_cdf], 
    'val_max': [x['val_max']['R2'] for x in performance_table_40_filt_rf_max]}
)

In [51]:
r2_table_20_rf

,train_cdf,train_max,val_cdf,val_max
0,0.618843,0.824500,-7.832996,-11.222197
1,0.559682,0.704008,-3.673632,-3.605653
2,0.157377,0.439014,-0.560989,-2.666519
3,0.800754,0.216729,-19.809484,-2.878900
4,0.789869,0.352543,-3.471427,-0.853664
5,0.074095,0.638983,-0.702390,-1.664192
6,0.397401,0.666194,-56.645389,-24.885813
7,0.257507,0.662528,-1.346550,-4.449953
8,0.495778,0.613392,-1.510879,-1.907528
9,0.580029,0.686712,-1.699891,-1.625588


In [52]:
r2_table_30_rf

,train_cdf,train_max,val_cdf,val_max
0,0.659693,0.137842,-12.026547,-0.707004
1,0.445199,0.581874,-1.700332,-2.367454
2,0.848920,1.000000,-15.673977,-24.467582
3,0.149253,1.000000,-2.803969,-51.317773
4,0.704242,0.396712,-6.211551,-1.661787
5,0.878077,1.000000,-68.409430,-26.699428
6,0.792232,0.496631,-240.686985,-34.866584
7,0.515839,0.667144,-3.813345,-8.077254
8,0.656628,0.524260,-4.160967,-1.840093
9,0.168234,0.540634,-0.734504,-3.532557


### Random Forest

In [53]:
# Train a RF model to predict CDF with the filtered data 
rf_20_cdf_filt_rf = [baseline.train_rf(data_dict_filt_cdf_rf[i]['Xtrain_20'], data_dict_filt_cdf_rf[i]['ytrain_cdf'], 42) for i in tqdm(range(len(data_dict_filt_cdf_rf)))]
rf_30_cdf_filt_rf = [baseline.train_rf(data_dict_filt_cdf_rf[i]['Xtrain_30'], data_dict_filt_cdf_rf[i]['ytrain_cdf'], 42) for i in tqdm(range(len(data_dict_filt_cdf_rf)))]
rf_40_cdf_filt_rf = [baseline.train_rf(data_dict_filt_cdf_rf[i]['Xtrain_40'], data_dict_filt_cdf_rf[i]['ytrain_cdf'], 42) for i in tqdm(range(len(data_dict_filt_cdf_rf)))]

# Train a RF model to predict max with the filtered data
rf_20_max_filt_rf = [baseline.train_rf(data_dict_filt_max_rf[i]['Xtrain_20'], data_dict_filt_max_rf[i]['ytrain_max'], 42) for i in tqdm(range(len(data_dict_filt_max_rf)))]
rf_30_max_filt_rf = [baseline.train_rf(data_dict_filt_max_rf[i]['Xtrain_30'], data_dict_filt_max_rf[i]['ytrain_max'], 42) for i in tqdm(range(len(data_dict_filt_max_rf)))]
rf_40_max_filt_rf = [baseline.train_rf(data_dict_filt_max_rf[i]['Xtrain_40'], data_dict_filt_max_rf[i]['ytrain_max'], 42) for i in tqdm(range(len(data_dict_filt_max_rf)))]

100%|██████████| 10/10 [01:42<00:00, 10.20s/it]


In [82]:
baseline.save_models(rf_20_cdf_filt_rf, '../../datas/proc/part1/rf_rf_20_cdf.pkl')
baseline.save_models(rf_30_cdf_filt_rf, '../../datas/proc/part1/rf_rf_30_cdf.pkl')
baseline.save_models(rf_40_cdf_filt_rf, '../../datas/proc/part1/rf_rf_40_cdf.pkl')
baseline.save_models(rf_20_max_filt_rf, '../../datas/proc/part1/rf_rf_20_max.pkl')
baseline.save_models(rf_30_max_filt_rf, '../../datas/proc/part1/rf_rf_30_max.pkl')
baseline.save_models(rf_40_max_filt_rf, '../../datas/proc/part1/rf_rf_40_max.pkl')

In [54]:
# Evaluate the model
rf_performance_table_20_filt_rf_cdf = metrics.evaluate_cdfmodel(rf_20_cdf_filt_rf, data_dict_filt_cdf_rf, 20)
rf_performance_table_30_filt_rf_cdf = metrics.evaluate_cdfmodel(rf_30_cdf_filt_rf, data_dict_filt_cdf_rf, 30)
rf_performance_table_40_filt_rf_cdf = metrics.evaluate_cdfmodel(rf_40_cdf_filt_rf, data_dict_filt_cdf_rf, 40)
rf_performance_table_20_filt_rf_max = metrics.evaluate_maxmodel(rf_20_max_filt_rf, data_dict_filt_max_rf, 20)
rf_performance_table_30_filt_rf_max = metrics.evaluate_maxmodel(rf_30_max_filt_rf, data_dict_filt_max_rf, 30)
rf_performance_table_40_filt_rf_max = metrics.evaluate_maxmodel(rf_40_max_filt_rf, data_dict_filt_max_rf, 40)

In [55]:
# Create a table storing the r2 values
rf_r2_table_20_rf = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_20_filt_rf_cdf], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_20_filt_rf_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_20_filt_rf_cdf], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_20_filt_rf_max]}
    )

rf_r2_table_30_rf = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_30_filt_rf_cdf], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_30_filt_rf_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_30_filt_rf_cdf], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_30_filt_rf_max]}
    )

rf_r2_table_40_rf = pd.DataFrame({
    'train_cdf': [x['train_cdf']['R2'] for x in rf_performance_table_40_filt_rf_cdf], 
    'train_max': [x['train_max']['R2'] for x in rf_performance_table_40_filt_rf_max], 
    'val_cdf': [x['val_cdf']['R2'] for x in rf_performance_table_40_filt_rf_cdf], 
    'val_max': [x['val_max']['R2'] for x in rf_performance_table_40_filt_rf_max]}
)

In [56]:
rf_r2_table_20_rf

,train_cdf,train_max,val_cdf,val_max
0,0.117918,0.126835,-1.397651,-0.578028
1,0.119025,0.129944,-1.215597,-0.511947
2,0.097494,0.423967,-0.034402,-0.119711
3,0.126871,0.469533,-0.586515,-0.700799
4,0.113250,0.476016,-0.106133,-0.207587
5,0.267743,0.139279,-1.093989,-0.507585
6,0.115335,0.130970,-5.988968,-1.300984
7,0.331804,0.131472,-0.504540,-0.356864
8,0.126622,0.856539,-0.178389,-0.105939
9,0.356031,0.444170,-0.259318,-0.352756


In [57]:
rf_r2_table_30_rf

,train_cdf,train_max,val_cdf,val_max
0,0.124868,0.120604,-0.860921,-0.490594
1,0.123478,0.130951,-0.532654,-0.439980
2,0.123921,0.153050,-0.045057,-0.421206
3,0.133294,0.144425,-0.731399,-0.431167
4,0.131636,0.440998,-0.181343,-0.308409
5,0.396446,0.142239,-0.837578,-0.494209
6,0.126821,0.481909,-5.638997,-1.836701
7,0.121518,0.137936,-0.201422,-0.380448
8,0.146811,0.139767,-0.128154,-0.157458
9,0.119262,0.444823,-0.257231,-0.335185


In [58]:
rf_r2_table_40_rf

,train_cdf,train_max,val_cdf,val_max
0,0.138744,0.124924,-0.847556,-0.505506
1,0.130619,0.122338,-0.515902,-0.435724
2,0.111039,0.132124,-0.057751,-0.023905
3,0.146905,0.142870,-1.056102,-0.437134
4,0.136353,0.630416,-0.195119,-0.237626
5,0.109924,0.459964,-0.467412,-0.510662
6,0.319648,0.144168,-6.623550,-1.223588
7,0.121313,0.144521,-0.179727,-0.368812
8,0.138001,0.145976,-0.187852,-0.132067
9,0.148486,0.134024,-0.248743,-0.304408


## Analysis

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
bestmodel_20 = rf_20_cdf_filt[rf_r2_table_20['val_cdf'].argmax()]

scaler = StandardScaler()
scaler.fit(data_dict_filt[rf_r2_table_20['val_cdf'].argmax()][f'Xtrain_{int(20)}'])
X_train_scaled = scaler.transform(data_dict_filt[rf_r2_table_20['val_cdf'].argmax()][f'Xtrain_{int(20)}'])
X_val_scaled = scaler.transform(data_dict_filt[rf_r2_table_20['val_cdf'].argmax()][f'Xvalid_{int(20)}'])

NameError: name 'rf_20_cdf_filt' is not defined

In [126]:
[r2_score(data_dict_filt[rf_r2_table_20['val_cdf'].argmax()]['yvalid_cdf'].iloc[:,i],bestmodel_20.predict(X_val_scaled)[:,i]) for i in range(15)]

[-0.00451929000559681,
 0.042471714600479604,
 -0.07192502851009852,
 -0.1333637002480632,
 0.15992191387868115,
 -0.267926220902712,
 -0.09920916065126906,
 -0.1355024969073333,
 -0.10479935682070152,
 -0.2036641568787334,
 0.06918699022306907,
 -0.06844787309190736,
 0.015830570915514763,
 -0.4003989947203872,
 -0.3897839393305935]